In [21]:
from boto3 import client
import json
import copy

with open('../config.json', 'r') as f:
    config = json.loads(f.read())['hitCreation']

if config['production']:
    endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
else:
    endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
    
cl = client('mturk', region_name='us-east-1', endpoint_url=endpoint_url)


In [22]:
def create_hit(task):
    quals = [
       {
           'QualificationTypeId': '00000000000000000071',
           'Comparator': 'EqualTo',
           'LocaleValues': [{
               'Country': 'US',
           }],
       },
       {
           'QualificationTypeId': '000000000000000000L0',
           'Comparator': 'GreaterThanOrEqualTo',
           'IntegerValues': [
               95
           ],
       },
    ]

    questionText = "<ExternalQuestion xmlns=\"http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/"
    questionText += "2006-07-14/ExternalQuestion.xsd\">\n<ExternalURL>" + task['taskUrl']
    questionText += "</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>"

    response = cl.create_hit(
        MaxAssignments=task['numAssignments'],
        AutoApprovalDelayInSeconds=604800,
        LifetimeInSeconds=task['lifetime'],
        AssignmentDurationInSeconds=task['duration'],
        Reward=task['rewardAmount'],
        Title=task['title'],
        Keywords=task['keywords'],
        Description=task['description'],
        Question=questionText,
        QualificationRequirements=quals,
    )

    print(response)

# JUST TO CREATE HIT

In [ ]:

if config['variants']: 
    print("creating " + str(len(config['variants'])) + " variants")
    for var in config['variants']: 
        task = copy.deepcopy(config)
        task.update(var)
        create_hit(task)
else: 
  print("creating " + str(config['numTasks']) + " tasks")
  for i in range(config['numTasks']):
      create_hit(config)

In [23]:
hits = cl.list_hits(MaxResults=9)['HITs']

hits

[{'AssignmentDurationInSeconds': 300,
  'AutoApprovalDelayInSeconds': 604800,
  'CreationTime': datetime.datetime(2018, 2, 9, 18, 7, 37, tzinfo=tzlocal()),
  'Description': 'Given two words, select the one you think describes a larger change.',
  'Expiration': datetime.datetime(2018, 2, 10, 18, 7, 37, tzinfo=tzlocal()),
  'HITGroupId': '3EWXWOT87MLR7GYP451ZBIS24W7P52',
  'HITId': '3P520RYKCHF2UL892LPJA5TG0S35UP',
  'HITReviewStatus': 'NotReviewed',
  'HITStatus': 'Assignable',
  'HITTypeId': '3XJ5RHVJOBCS36D8T5MUWHH8C0IXL0',
  'Keywords': 'words, comparison, ranking, fun, challenge, quick',
  'MaxAssignments': 10,
  'NumberOfAssignmentsAvailable': 9,
  'NumberOfAssignmentsCompleted': 1,
  'NumberOfAssignmentsPending': 0,
  'QualificationRequirements': [{'Comparator': 'EqualTo',
    'LocaleValues': [{'Country': 'US'}],
    'QualificationTypeId': '00000000000000000071',
    'RequiredToPreview': False},
   {'Comparator': 'GreaterThanOrEqualTo',
    'IntegerValues': [95],
    'Qualificatio

In [24]:
[hit['NumberOfAssignmentsAvailable'] for hit in hits]

[9, 9, 10, 10, 10, 9, 10, 9, 9]

In [25]:
import datetime

def summarize_hits(hits): 
    print(len(hits))
    ret = ""
    for hit in hits: 
        expiration = hit['Expiration'].replace(tzinfo=None)
        is_expired = expiration < datetime.datetime.now()
        description = ("Title: {title}\n" 
        "ID: {hid}\n"
        "\tAssignments left: {left}\n"
        "\tAssignments completed: {complete}\n"
        "\tAssignments pending: {pending}\n"
        "\tExpired: {exp}\n\n").format(
            title=hit['Title'], 
            hid=hit['HITId'], 
            left=hit['NumberOfAssignmentsAvailable'], 
            complete=hit['NumberOfAssignmentsCompleted'], 
            pending=hit['NumberOfAssignmentsPending'],
            exp=str(is_expired)
        )
        ret += description
    print(ret)
        

In [26]:
summarize_hits(hits)

pass

9
Title: Which word is more intense?
ID: 3P520RYKCHF2UL892LPJA5TG0S35UP
	Assignments left: 9
	Assignments completed: 1
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 329E6HTMSWBLDAGTUOXOPLMYK5E3K0
	Assignments left: 9
	Assignments completed: 1
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 3JVP4ZJHDP1YQNR8BYUTA79EPP50IR
	Assignments left: 10
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 3J9UN9O9J31ULYBETGFTM8OE24U0J3
	Assignments left: 10
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 3VGZ74AYTGP4OOGTXQNAKWYZJ1ACG4
	Assignments left: 10
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 386T3MLZLN451HFV2CI2XIF0MT080R
	Assignments left: 9
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Which word is more intense?
ID: 3UQ1LLR2

In [7]:
def see_assignments(hits):
    ret = ""
    for hit in hits: 
        hid = hit['HITId']
        title =  hit['Title']
        name = "HIT %s: %s" % (hid, title)
        ret += name + "\n"
        assignments = cl.list_assignments_for_hit(HITId=hid)['Assignments']
        for a in assignments: 
            desc = "\tAssignment {aid}\n\t\tStatus: {status}\n".format(aid=a['AssignmentId'], status=a['AssignmentStatus'])
            ret += desc
    print(ret)
        
        

In [33]:
see_assignments(hits)

pass

HIT 3P520RYKCHF2UL892LPJA5TG0S35UP: Which word is more intense?
	Assignment 379J5II41PQ2UYC2UAWNCWZRBL4LE9
		Status: Approved
HIT 329E6HTMSWBLDAGTUOXOPLMYK5E3K0: Which word is more intense?
	Assignment 3KOPY89HM9CTPA85253EDQI2G79J3E
		Status: Approved
HIT 3JVP4ZJHDP1YQNR8BYUTA79EPP50IR: Which word is more intense?
HIT 3J9UN9O9J31ULYBETGFTM8OE24U0J3: Which word is more intense?
HIT 3VGZ74AYTGP4OOGTXQNAKWYZJ1ACG4: Which word is more intense?
HIT 386T3MLZLN451HFV2CI2XIF0MT080R: Which word is more intense?
	Assignment 3VD82FOHKRYV3LVLBB6KEP3ILEXOC1
		Status: Approved
HIT 3UQ1LLR26AHWH1XWO3OZRTDJFFBAL6: Which word is more intense?
HIT 311HQEI8RSPY3OG2W13OPHMRTGD7ZV: Which word is more intense?
	Assignment 3DR23U6WE6OQDBK73JYEYN985YRTEL
		Status: Approved
HIT 3UUIU9GZC5ENU8TK2GZ9597EHEW5TY: Which word is more intense?
	Assignment 35GMH2SV3FRA0JFZ72CF4D3L6QAOEU
		Status: Submitted



In [34]:
def approve_all(hits): 
    num_approved = 0
    for hit in hits: 
        assignments = cl.list_assignments_for_hit(HITId=hit['HITId'])['Assignments']
        for a in assignments: 
            if a['AssignmentStatus'] != 'Approved':
                print("Approving assignment")
                num_approved += 1
                cl.approve_assignment(AssignmentId=a['AssignmentId'])
    print("Approved %d assignments" % num_approved)

In [35]:
approve_all(hits)

Approving assignment
Approved 1 assignments


In [29]:
from bs4 import BeautifulSoup as bs 
import pprint

def pretty_print(obj):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(obj)

def get_assignment_content(hits, download_path="", should_print=False): 
    all_responses = []
    for hit in hits: 
        hitid = hit['HITId']
        assignments = cl.list_assignments_for_hit(HITId=hit['HITId'])['Assignments']
        for a in assignments:
            a_xml = a['Answer']
            #print(a_xml)
            soup = bs(a_xml, "lxml")
            answers = soup.find_all("answer")
            #print(answers)
            results = {'HITId': hitid}
            for ans in answers: 
                identifier = ans.find('questionidentifier').string
                answer = ans.find('freetext').string
                #print(identifier)
                #print(answer)
                try: 
                    results[identifier] = json.loads(answer)
                except:
                    results[identifier] = answer
            all_responses.append(results)
    if should_print: 
        pretty_print(all_responses)
    if download_path: 
        with open(download_path, 'w') as outfile: 
            json.dump(all_responses, outfile)
    return all_responses
            

In [36]:
get_assignment_content(hits, download_path='../results/results.json', should_print=True)

pass

[   {   'HITId': '3P520RYKCHF2UL892LPJA5TG0S35UP',
        'feedback': None,
        'results': {   'inputs': [   ['consistent', 'rapid'],
                                     ['small', 'dramatic'],
                                     ['gradual', 'small'],
                                     ['huge', 'tiny'],
                                     ['steep', 'sharp'],
                                     ['steep', 'negligible'],
                                     ['dramatic', 'gentle'],
                                     ['modest', 'negligible'],
                                     ['steep', 'noticeable'],
                                     ['noticeable', 'substantial'],
                                     ['negligible', 'slight'],
                                     ['noticeable', 'gentle'],
                                     ['huge', 'considerable'],
                                     ['considerable', 'dramatic'],
                                     ['sharp', 'substantia

                                      'slight',
                                      'small',
                                      'steady',
                                      'slow',
                                      'sharp',
                                      'small',
                                      'sudden',
                                      'gentle',
                                      'slow',
                                      'slow',
                                      'modest',
                                      'steep',
                                      'small',
                                      'huge',
                                      'steady',
                                      'slow',
                                      'consistent',
                                      'large',
                                      'consistent',
                                      'significant',
                                      'gent

[]